# Data Preparation (PDF → JSON)

**Goal:**  
Convert all Swiss rental-law PDFs (OR, VMWG, StGB) into clean, structured JSON files —  
where **each JSON = exactly one legal article**.

This makes later retrieval and referencing much easier and more accurate.

**Context:**  
- Splitting at *article-level granularity* instead of page chunks.
- Adding metadata (law name, article number, source).
- Keeping a clean and reproducible data pipeline.

## ⚙️ Imports and Setup

In [1]:
import re, json
from pathlib import Path
import pymupdf
from tqdm import tqdm

# Paths
DATA_RAW = Path("../data/raw")     # PDFs go here
DATA_JSON = Path("../data/json")   # Will hold one JSON per article
DATA_JSON.mkdir(parents=True, exist_ok=True)


### Explanation
We’ll use:
- **PyMuPDF (fitz)** to extract text page by page.  
- **Regex** to detect “Art. XXX” headers.  
- **tqdm** for nice progress bars.  

We’ll store results as JSON so each file can be directly embedded later.


### 🧩 Helper Functions

In [2]:
# --- Cleaning & Splitting ---
ART_HEADER = re.compile(r"(?m)^\s*(Art\.\s*\d+[a-zA-Z]*\b[^\n]*)\s*$")

def clean_text(t: str) -> str:
    """Normalize whitespace and remove artifacts."""
    t = t.replace("\x0c", " ").replace("\u00ad", "")
    t = re.sub(r"[ \t]+", " ", t)
    t = re.sub(r"\s+\n", "\n", t)
    t = re.sub(r"\n\s+", "\n", t)
    return t.strip()

def read_pdf_text(pdf_path: Path) -> str:
    """Extract all text from a PDF using PyMuPDF."""
    pages = []
    with pymupdf.open(pdf_path) as doc:
        for p in doc:
            pages.append(p.get_text("text"))
    return clean_text("\n".join(pages))

def split_articles(full_text: str):
    """Split a document into (header, body) per article."""
    headers = list(ART_HEADER.finditer(full_text))
    articles = []
    for i, m in enumerate(headers):
        start = m.start()
        end = headers[i+1].start() if i+1 < len(headers) else len(full_text)
        block = full_text[start:end].strip()
        header_line = m.group(1).strip()
        body = block[len(header_line):].strip()
        articles.append((header_line, body))
    return articles

def parse_article_number(header_line: str):
    m = re.search(r"Art\.\s*(\d+[a-zA-Z]*)", header_line)
    return m.group(1) if m else None


### Explanation
Each article in Swiss laws starts with `Art.` followed by a number or letter.  
This regex isolates those headers and splits the PDF into article blocks.  
We also extract the article number (e.g. `269d`, `325bis`) for metadata.


## 📄 Process PDFs → Save JSON

In [3]:
def detect_law_tag(stem: str) -> str:
    s = stem.upper()
    if "OR" in s: return "OR"
    if "VMWG" in s: return "VMWG"
    if "STGB" in s or "STG" in s: return "StGB"
    if "SRL" in s: return "SRL"
    return stem

def ingest_pdf(pdf_path: Path):
    law = detect_law_tag(pdf_path.stem)
    text = read_pdf_text(pdf_path)
    articles = split_articles(text)

    out_dir = DATA_JSON / law
    out_dir.mkdir(parents=True, exist_ok=True)

    for header, body in articles:
        art_nr = parse_article_number(header) or "NA"
        payload = {
            "law": law,
            "article": art_nr,
            "header": header,
            "text": body,
            "source": pdf_path.name
        }
        out_fp = out_dir / f"{law}_Art_{art_nr}.json"
        out_fp.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")

    return len(articles)


### ▶️ Run Conversion

In [4]:
pdfs = sorted(DATA_RAW.glob("*.pdf"))
print("Found PDFs:", [p.name for p in pdfs])

total_articles = 0
for pdf in tqdm(pdfs, desc="Processing PDFs"):
    total_articles += ingest_pdf(pdf)

print(f"✅ Done! Created ~{total_articles} article JSON files.")


Found PDFs: ['OR.pdf', 'SRL.pdf', 'STGB.pdf', 'VMWG.pdf']


Processing PDFs: 100%|██████████| 4/4 [00:00<00:00, 62.17it/s]

✅ Done! Created ~119 article JSON files.


### Explanation
Each PDF is scanned and split into articles.
Every JSON file now represents **exactly one article** (e.g. `OR_Art_269d.json`).
We'll use these later to build embeddings with ChromaDB.


## 🗒️ Process Lucerne forms

In [5]:
FORMS_DIR = DATA_JSON / "LU_FORMS"
FORMS_DIR.mkdir(parents=True, exist_ok=True)

FORMS = [
    {
        "id": "lu_mietvertragsaenderung",
        "name": "Formular Mietvertragsänderung",
        "description": (
            "Offizielles Luzerner Formular für Mietzinserhöhungen und andere einseitige Vertragsänderungen des Vermieters."
        ),
        "used_when": (
            "PERS: landlord | "
            "POS: mietzinserhöhung, mietzins erhöhen, erhöhung, mietzinsanpassung, energetische verbesserung, vertragsänderung, vermieter ändert vertrag | "
            "NEG: anfangsmietzins, neuabschluss, kündigung"
        ),
        "download_url": "https://gerichte.lu.ch/-/media/Gerichte/Dokumente/rechtsgebiete/formulare/Mietrecht/Mietvertragsaenderung_pdf.pdf",
    },
    {
        "id": "lu_anfangsmietzins",
        "name": "Mitteilung des Anfangsmietzinses",
        "description": (
            "Formular nach Art. 269d OR zur Mitteilung des Anfangsmietzinses, wenn die Formularpflicht im Kanton Luzern gilt."
        ),
        "used_when": (
            "PERS: landlord | "
            "POS: anfangsmietzins, neuvermietung, neuer mietvertrag, erstvermietung, vermieter teilt mietzins mit | "
            "NEG: mietzinserhöhung, kündigung"
        ),
        "download_url": "https://gerichte.lu.ch/-/media/Gerichte/Dokumente/rechtsgebiete/formulare/Mietrecht/Mitteilung_des_Anfangsmietzinses.pdf",
    },
    {
        "id": "lu_kuendigung",
        "name": "Kündigungsformular Wohn- und Geschäftsräume",
        "description": (
            "Offizielles Kündigungsformular für Wohn- und Geschäftsräume im Kanton Luzern."
        ),
        "used_when": (
            "PERS: both | "
            "POS: kündigung, mietvertrag beenden, auszug, beendigung des vertrags | "
            "NEG: anfangsmietzins, mietzinserhöhung"
        ),
        "download_url": "https://gerichte.lu.ch/-/media/Gerichte/Dokumente/rechtsgebiete/formulare/Mietvertrag_Kuendigung_pdf.pdf",
    },
    {
        "id": "lu_schlichtungsgesuch",
        "name": "Schlichtungsgesuch Miete/Pacht",
        "description": (
            "Gesuch an die Schlichtungsbehörde Miete/Pacht Luzern zur Einleitung eines Verfahrens bei Streitigkeiten (z.B. Mietzinserhöhung, Kündigung)."
        ),
        "used_when": (
            "PERS: both | "
            "POS: schlichtung, streitigkeit, anfechtung, konflikt, "
            "mietzinserhöhung anfechten, kündigung anfechten, mietstreit | "
            "NEG: rein formale kündigung ohne streit"
        ),
        "download_url": "https://gerichte.lu.ch/-/media/Gerichte/Dokumente/rechtsgebiete/formulare/schlichtungsgesuch_sbm_pdf.pdf",
    },
    {
        "id": "lu_vollmacht_schlichtung",
        "name": "Vollmacht für Schlichtungsverhandlung",
        "description": (
            "Vollmacht gemäss Art. 204 Abs. 3 lit. d ZPO für Vertretung in der Schlichtungsverhandlung."
        ),
        "used_when": (
            "PERS: both | "
            "POS: vollmacht, vertretung, anwalt vertritt, dritte person vertritt, "
            "schlichtungsverhandlung | "
            "NEG: keine vertretung nötig"
        ),
        "download_url": "https://gerichte.lu.ch/-/media/Gerichte/Dokumente/Organisation/Fromular_Gesuch_um_Entbindung_von_der_persnlichen_Erscheinungspflicht.docx",
    },
]

for f in FORMS:
    payload = {
        # keep the existing schema so the embedding script still works
        "law": "LU_FORMS",
        "article": f["id"],
        "header": f["name"],
        "text": f["description"] + "\n\nVerwendet wenn: " + "; ".join(f["used_when"]),
        "source": f["download_url"],

        # extra fields your app can use later
        "form_name": f["name"],
        "form_url": f["download_url"],
        "used_when": f["used_when"],
        "jurisdiction": "Kanton Luzern",
    }

    out_fp = FORMS_DIR / f"{f['id']}.json"
    out_fp.write_text(json.dumps(payload, ensure_ascii=False, indent=2), encoding="utf-8")

print(f"✅ Added {len(FORMS)} Lucerne form metadata files.")

✅ Added 5 Lucerne form metadata files.


### Explanation

This step adds the Lucerne rental forms as small metadata JSON files. Each file represents **one form** with its name, purpose and download link. \
They are indexed like the law articles above so the model can show the correct form and a download button when needed.

## 👀 Quick Inspection

In [6]:
samples = list((DATA_JSON / "OR").glob("*.json"))[:3]
for s in samples:
    print("File:", s.name)
    data = json.loads(s.read_text(encoding="utf-8"))
    print(f"Header: {data['header']}")
    print(f"Excerpt: {data['text'][:250]}...\n")


File: OR_Art_270b.json
Header: Art. 270b, Anfechtung von Mietzinserhöhungen und andern einseitigen Vertragsänderungen
Excerpt: 1 Der Mieter kann eine Mietzinserhöhung innert 30 Tagen, nachdem sie ihm mitgeteilt worden
ist, bei der Schlichtungsbehörde als missbräuchlich im Sinne der Artikel 269 und 269a anfechten.
2 Absatz 1 gilt auch, wenn der Vermieter sonstwie den Mietvert...

File: OR_Art_268b.json
Header: Art. 268b, Geltendmachung
Excerpt: 1 Will der Mieter wegziehen oder die in den gemieteten Räumen befindlichen Sachen fortschaffen,
so kann der Vermieter mit Hilfe der zuständigen Amtsstelle so viele Gegenstände
zurückhalten, als zur Deckung seiner Forderung notwendig sind.
2 Heimlich ...

File: OR_Art_266.json
Header: Art. 266, Beendigung des Mietverhältnisses, Ablauf der vereinbarten Dauer
Excerpt: 1 Haben die Parteien eine bestimmte Dauer ausdrücklich oder stillschweigend vereinbart, so
endet das Mietverhältnis ohne Kündigung mit Ablauf dieser Dauer.
2 Setzen die Parteien das Mi

### Explanation
We quickly check that:
- Articles are correctly separated.  
- Text doesn’t include the next article.  
- Metadata (law, article number) is stored correctly.


# ✅ Summary
We now have a clean, article-level dataset ready for indexing.

**Next notebook: `2_Indexing_and_Retrieval.ipynb`**
We'll:
- Load all JSONs,
- Embed them with Sentence Transformers,
- Store them in a persistent ChromaDB collection for fast semantic search.

**Benefits of this structure**
- Easier to debug and explain
- Perfect granularity (one legal article per data point)
- Can easily add new laws or update existing ones
